In [58]:
%reload_ext autoreload
%autoreload 1
import polars as pl
from sklearn.pipeline import Pipeline
import auxiliary.transformers as tr
from auxiliary.transformers import PolarsColumnTransformer as PCT
from lightgbm import LGBMClassifier
import auxiliary.tuning as tunes
from ray import tune
%aimport auxiliary.transformers
%aimport auxiliary.tuning

In [59]:
train_data=pl.read_parquet('temp/application_train_filtered.parquet')
id_and_target=['SK_ID_CURR','TARGET']
X_train=train_data.drop(columns=id_and_target)
y_train=train_data['TARGET']

In [60]:
bool_features = []
for feature in X_train.columns:
    if train_data[feature].drop_nulls().n_unique() == 2:
        bool_features.append(feature)

cat_features = [
    feature
    for feature in X_train.select(pl.col(pl.Utf8)).columns
    if feature not in bool_features
]

In [61]:
# for feature in cat_features:
#     display(X_train[feature].value_counts())

In [62]:
preprocessing=Pipeline([])
encoders=tr.PolarsColumnTransformer([])
for feature in bool_features:
    encoders.steps[feature]=PCT.Step(feature,tr.PolarsOneHotEncoder(drop=True),feature)
for feature in cat_features:
    encoders.steps[feature]=PCT.Step(feature,tr.TargetMeanOrderedLabeler(how='label'),feature)
preprocessing.steps.append(('encoders',encoders))

In [63]:
model_lgb=LGBMClassifier(n_jobs=1,verbosity=-1,force_col_wise=True)
full_pipeline=Pipeline([('preprocessing', preprocessing),('model',model_lgb)])

In [64]:
scores

[0.772704068028358,
 0.7740375742673732,
 0.7676323685114121,
 0.7766943413310012,
 0.7756533490237596]

In [65]:
models = tunes.Models()
search_space_lgbm = {
    "model__max_depth": tune.randint(5, 50),
    "model__num_leaves": tune.randint(10, 3051),
    "model__n_estimators": tune.randint(10, 251),
    "model__learning_rate": tune.loguniform(0.001, 0.1),
    "model__bagging_freq": tune.randint(0, 11),
    "model__colsample_bytree": tune.uniform(0.2, 1.0),
    "model__subsample": tune.uniform(0.2, 1.0),
    "model__reg_alpha": tune.loguniform(0.001, 100),
    "model__reg_lambda": tune.loguniform(0.001, 100),
    "model__boosting_type": tune.choice(["gbdt", "dart", "rf"]),
    "model__class_weight": tune.choice(["balanced", None]),
    "model__max_bin": tune.randint(5, 201),
}

models.add_model(
    "lgbm_grade_single", full_pipeline, search_space_lgbm,
)


In [66]:
models.tune_all(X_train,y_train,sample_size=10000,metric='roc_auc')

(TrainableCV pid=19742) Step 0 F-1 Score: 0.7558275712404214
(TrainableCV pid=19742) Step 1 F-1 Score: 0.6895991847826087
(TrainableCV pid=19742) Step 2 F-1 Score: 0.7443478260869565
(TrainableCV pid=19742) Step 3 F-1 Score: 0.7064970714223272
(TrainableCV pid=19791) Step 0 F-1 Score: 0.772737817289272
(TrainableCV pid=19742) Step 4 F-1 Score: 0.7273815979754157
(TrainableCV pid=19791) Step 4 F-1 Score: 0.7493707937037656 [repeated 4x across cluster]
(TrainableCV pid=19896) Step 0 F-1 Score: 0.7563944264846744 [repeated 2x across cluster]
(TrainableCV pid=19998) Step 1 F-1 Score: 0.7061073369565217 [repeated 4x across cluster]
(TrainableCV pid=19945) Step 2 F-1 Score: 0.7636005434782609 [repeated 7x across cluster]
(TrainableCV pid=20113) Step 1 F-1 Score: 0.6769497282608696 [repeated 6x across cluster]
(TrainableCV pid=20059) Step 4 F-1 Score: 0.737638355859614 [repeated 8x across cluster]
(TrainableCV pid=19840) Step 4 F-1 Score: 0.7328828077201179 [repeated 4x across cluster]
(Train

2023-10-30 14:21:01,334	INFO tune.py:1143 -- Total run time: 560.41 seconds (559.52 seconds for the tuning loop).


lgbm_grade_single tuned.


In [69]:
scores = []
for train_index, test_index in StratifiedKFold(5).split(X_train, y_train):
    models.models["lgbm_grade_single"].pipeline.fit(
        X_train[train_index], y_train[train_index]
    )
    scores.append(
        roc_auc_score(
            y_train[test_index],
            models.models["lgbm_grade_single"].pipeline.predict_proba(
                X_train[test_index]
            )[:,1],
        )
    )

In [70]:
scores

[0.7613917144800472,
 0.7621113184868856,
 0.7550835899762174,
 0.7663219521634728,
 0.7639852457126839]